In [ ]:
from tokenizer import Tokenizer
import torch
import torch.nn as nn

In [10]:
obj = Tokenizer()
obj.character_tokenizer()
data=obj.encoder(s=obj.data)

In [11]:
n=int(0.9*len(data))
traindata = data[:n]
intrimdata = data[n:]

In [ ]:
block_size = 128  # context length
batch_size = 64   # number of sequences in a batch

data = torch.tensor(traindata) 
ix = torch.randint(len(data) - block_size, (batch_size,))
x = torch.stack([data[i:i+block_size] for i in ix])
y = torch.stack([data[i+1:i+block_size+1] for i in ix])


TypeError: expected Tensor as element 0 in argument 0, but got list

In [26]:
import torch
import torch.nn as nn

vocab_size = 5000
embed_dim = 128
max_len = 512   # max sequence length

# Token embeddings
tok_emb = nn.Embedding(vocab_size, embed_dim)

# Positional embeddings
pos_emb = nn.Embedding(max_len, embed_dim)

# Example input (batch of 2 sentences, each length 5)
x = torch.randint(0, vocab_size, (2, 5))

# Positions (0..4)
positions = torch.arange(0, x.size(1)).unsqueeze(0)

# Final embeddings
out = tok_emb(x) + pos_emb(positions)
print(out.shape)  # (2, 5, 128)


torch.Size([2, 5, 128])
